In [7]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [8]:
from numpy import array, delete
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import mean_absolute_error
from itertools import combinations

In [9]:
resturants = pd.read_csv('./data/restaurants.csv')
data_size = len(resturants)

# Clean data

Check if there are any missing data. Each column is categorical so check that the number of unique categroies makes sense.

In [10]:
def explore():
    for column in resturants.columns:
        print(column, ' : ', resturants[column].unique(), '\n')

In [11]:
print('Missing values: {missing}\n'.format(missing=resturants.isnull().values.any()))
explore()

Missing values: False

age  :  ['35_49' '50_64' '20_34' 'under_19' '65_and_over'] 

gender  :  ['female' 'male'] 

budget  :  ['under_20' '20_to_30' 'over_50' '30_to_50'] 

price  :  ['20_to_30' '30_to_50' 'under_20' 'over_50'] 

cuisine_type  :  ['Latin American/Mexican' 'American' 'Asian' 'Bars/Pubs'
 'Deli/Sandwiches/Fast Food' 'Continental' 'African' 'Breakfast/Brunch'
 'Seafood' 'Mediterranean' 'Vegetarian/vegan' 'Cafe'] 

rating  :  ['very good' 'dislike' 'satisfactory' 'excellent'] 



# Data setup

Tell pandas that our data is categorical. It makes senese to assign an order to some of the features, so tell pandas the order too. We will shorten the cuisine names to make visualization easier.

In [12]:
resturants = resturants.astype('category')
resturants.age = resturants.age.cat.reorder_categories(
    ['under_19', '20_34', '35_49', '50_64', '65_and_over'], ordered=True)
resturants.budget = resturants.budget.cat.reorder_categories(
    ['under_20', '20_to_30', '30_to_50', 'over_50'], ordered=True)
resturants.price = resturants.price.cat.reorder_categories(
    ['under_20', '20_to_30', '30_to_50', 'over_50'], ordered=True)
resturants.rating = resturants.rating.cat.reorder_categories(
    ['dislike', 'satisfactory', 'very good', 'excellent'], ordered=True)

full_cuisine_names = {k:v for k,v in enumerate(resturants.cuisine_type.cat.categories)}
resturants.cuisine_type.cat.rename_categories(
    [cat[0:11] for cat in resturants.cuisine_type.cat.categories]);

# Visualize the Data

Since all of the data is categorical the primary way to visualize is to compare histograms over certain subsets of the data. Lets look at the histograms of all of our features to get the lay of the land.

In [13]:
def explore_visual():
    resturants.describe(include='all')

    fig, axes = plt.subplots(2, 3)
    for i, column in enumerate(resturants.columns):
        ax = sns.countplot(resturants[column], ax=axes[i % 2, i % 3])
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

In [14]:
explore_visual()

## More informative charts

I'd like to see the histograms given as a percentage of the total subset and report just how much of our data set I am looking at. I'll get ideas where to go from this info, we can alway do a hypotheses test to make sure the interpretation is valid.

In [15]:
def percentplot(data, **kwargs):
    ax = sns.barplot(
        x=data,
        y=data,
        orient='v',
        ci=None,
        estimator=lambda x: len(x) * 100.0 / len(data),
        alpha=.5,
        **kwargs)

    float_percent = len(data) * 100.0 / data_size
    if ax.texts:
        float_percent = len(data) * 100.0 / data_size
        for text in ax.texts:
            float_percent += float(text.get_text()[:-1])
        ax.texts = []

    percent = '{:.2g}%'.format(float_percent)

    ax.text(.5, .95, percent, transform=ax.transAxes)
    return ax

In [16]:
def facetplot(facet):
    # price vs age
    # age vs rating
    for col in resturants.columns:
        if col != facet:
            g = sns.FacetGrid(resturants, col=facet)
            g.map(percentplot, col)

In [17]:
facetplot('age')

## FacetGrid

Although it's a bit of data overload, we can use facet plot to view the relationships between even more features.

In [18]:
def bigfacet(data, facet, col, row=None, hue=None):
    if hue and row:
        g = sns.FacetGrid(data, row=row, hue=hue, col=col, palette='Set1')
        g.map(percentplot, facet)
        g.set_titles('{row_name:.1s}|{col_name:.8s}')
        g.add_legend()
    elif row:
        g = sns.FacetGrid(data, row=row, col=col, palette='Set1')
        g.map(percentplot, facet)
        g.set_titles('{row_name:.1s}|{col_name:.8s}')
    elif hue:
        g = sns.FacetGrid(data, hue=hue, col=col, palette='Set1')
        g.map(percentplot, facet)
        g.set_titles('{col_name:.8s}')
        g.add_legend()

    x_tick_labels = []
    for cat in data[facet].cat.categories:
        x_tick_labels.append(cat[0])

    g.set_xticklabels(x_tick_labels)

In [19]:
bigfacet(resturants, 'rating','cuisine_type', row='age')

/Users/admin/miniconda3/lib/python3.6/site-packages/matplotlib/tight_layout.py:198: UserWarning: tight_layout cannot make axes width small enough to accommodate all axes decorations
  warnings.warn('tight_layout cannot make axes width small enough '


In [20]:
bigfacet(resturants, 'rating', 'cuisine_type', hue='gender')

/Users/admin/miniconda3/lib/python3.6/site-packages/matplotlib/tight_layout.py:198: UserWarning: tight_layout cannot make axes width small enough to accommodate all axes decorations
  warnings.warn('tight_layout cannot make axes width small enough '


# We could automate

All we are relly doing here is searching though histograms to see if we can spot any differences in the relative distributions. We could choose a reasonable p-value and compute chi square stastics parewise. 

In [21]:
def chi_square(row, col, data, lambda_=1):
    row_vars = data[row].cat.categories
    col_vars = data[col].cat.categories
    observations = []

    for var1 in row_vars:
        col_obs = []
        for var2 in col_vars:
            col_obs.append(
                len(
                    data.query('{row} == @var1'.format(row=row)).query(
                        '{col} == @var2'.format(col=col))))
        observations.append(col_obs)

    obs = array(observations)
    zeros, dropped = [], 0

    for index, elem in enumerate(obs.sum(axis=0)):
        if elem == 0:
            zeros.append(index)
            dropped += 1
    obs = delete(obs, zeros, axis=1)
    return chi2_contingency(obs, lambda_=lambda_)

In [22]:
for row, col in combinations(resturants.columns, 2):
    pval = chi_square(row, col, resturants)[1]
    if pval < .1:
        print(row, 'vs', col,':', pval)

age vs gender : 6.712210367540001e-21
age vs budget : 0.0004905628509387912
age vs rating : 0.0
gender vs budget : 0.005307531562409769
gender vs rating : 2.695313473992979e-13
budget vs rating : 0.0
price vs cuisine_type : 0.0
price vs rating : 0.0
cuisine_type vs rating : 0.0


We can investigate the relationship between gender, age and cuisine type further.

In [23]:
useful_features = []
for age in resturants.age.cat.categories:
    print('-----{age}-----'.format(age=age))
    for cat in resturants.cuisine_type.cat.categories:
        pval = chi_square('gender', 'rating', resturants.query('cuisine_type == @cat and age == @age'))[1]
        if pval < .1:
            print(cat, ':', pval)
            useful_features.append((age,cat))
    print('\n')

-----under_19-----
Vegetarian/vegan : 0.00011186598586295131


-----20_34-----
African : 1.403702577113272e-15


-----35_49-----
African : 4.2471150893987534e-07
Bars/Pubs : 1.1357149180974728e-64
Deli/Sandwiches/Fast Food : 0.09400665534758654
Seafood : 2.6291468824417726e-53


-----50_64-----
Vegetarian/vegan : 6.6713641069991425e-06


-----65_and_over-----
Cafe : 0.03460917828870051
Latin American/Mexican : 0.012992500968899366
Mediterranean : 0.01107559672940161
Vegetarian/vegan : 3.2006722876321e-05




The cuisine types above are those which have a p-value less than 0.1, indicating that gender does make a defference in ratings given to these cuisines in the given age bracket. We could process these further automatically and use this same process across the whole data set to help with feature selection.

Or, use this infomation to drill down to graphs that display usefull information about our population. As an example lets check how males and females aged 35 to 49 differ on their opinion of Bars/Pubs.

In [24]:
row, col = useful_features[4]
ax = sns.countplot(x='rating', hue = 'gender', data=resturants.query('cuisine_type == @col and age == @row'), palette='Set1')
ax.set_xlabel(row + ' | '+col)

Text(0.5, 0, '35_49 | Deli/Sandwiches/Fast Food')

# A telling image
With automated hypothesis testing we can qickly find relationships in the data. And, since there arent that many features in our data set we can visualize those relationships too.

In [25]:
bigfacet(resturants, 'rating','cuisine_type', row='age', hue='gender')

/Users/admin/miniconda3/lib/python3.6/site-packages/matplotlib/tight_layout.py:198: UserWarning: tight_layout cannot make axes width small enough to accommodate all axes decorations
  warnings.warn('tight_layout cannot make axes width small enough '


# Predictive models
Can we predict what rating a person will assign to a given cuisine? A decision tree seems like a good first pass with this low dimensional catigorical data.

In [26]:
def predict_with_tree(features, label, tree_type='regressor', **kwargs):
    feats = []
    for col in features:
        feats.append(resturants[col].cat.codes.values)
    X = array(feats)
    X = X.T
    y = resturants[label].cat.codes.values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.40)
    if tree_type == 'regressor':
        clf = tree.DecisionTreeRegressor(**kwargs)
    elif tree_type == 'classifier':
        clf = tree.DecisionTreeClassifier(**kwargs)
    else:
        raise ValueError('tree_type must be one of [regressor, classifier]')
    clf.fit(X_train, y_train)
    return clf, X_test, y_test

In [27]:
def test_model(model, num_tests, *args, **kwargs):
    error = []
    for i in range(num_tests):
        clf, X_test, y_test = model(*args, **kwargs)
        error.append(mean_absolute_error(y_test, clf.predict(X_test)))
    error = array(error)
    return error.mean()

In [294]:
for r in range(1,6):
    for features in combinations(resturants.columns[0:5], r):
        error = test_model(predict_with_tree, 10, features, 'rating', tree_type='regressor')
        if error < .5:
            print(features, ':', error,'\n')

('age', 'gender', 'cuisine_type') : 0.48662250794186973 

('age', 'budget', 'cuisine_type') : 0.30678517227011326 

('age', 'price', 'cuisine_type') : 0.4267288945461364 

('age', 'gender', 'budget', 'cuisine_type') : 0.2687036719708614 

('age', 'gender', 'price', 'cuisine_type') : 0.3922324473950226 

('age', 'budget', 'price', 'cuisine_type') : 0.13384768080845033 

('age', 'gender', 'budget', 'price', 'cuisine_type') : 0.08497379996237872 



In [295]:
bigfacet(resturants, 'rating','cuisine_type', row='budget', hue='gender')

/Users/admin/miniconda3/lib/python3.6/site-packages/matplotlib/tight_layout.py:198: UserWarning: tight_layout cannot make axes width small enough to accommodate all axes decorations
  warnings.warn('tight_layout cannot make axes width small enough '


In [334]:
#add plot budget vs rating
g = sns.FacetGrid(resturants, col='budget')
g.map(percentplot, 'rating')
g.set_titles('{col_name:.8s}')

In [336]:
g = sns.FacetGrid(resturants, col='price')
g.map(percentplot, 'rating')
g.set_titles('{col_name:.8s}')

# Over budget

When I was first exploring this data set I came across something slightly surprising.

In [297]:
resturants['stars'] = resturants.rating.cat.codes
resturants['over_budget'] = pd.Categorical(
    resturants.budget < resturants.price)

len(resturants[resturants.budget < resturants.price])/data_size*100.0

36.55486436221381

About 37% of the reviewers in this data set are going over budget. Further, reviewers who do go over budget are rating resturants nearly an average of one star lower than those who do not.

In [298]:
print('Over budget:',resturants.query('over_budget == True').stars.mean())
print('At or under budget: ', resturants.query('over_budget == False').stars.mean())

Over budget: 1.4642105263157894
At or under budget:  2.446275144041241


The decision tree using all features was able to learn this relationship. But, using this feature alone we can do just as well as the model which uses all features, and we end up with a more interpretable result.

In [299]:
test_model(predict_with_tree, 10, ['age', 'gender','over_budget', 'cuisine_type'], 'rating', tree_type='regressor')

0.08378750406667444

In [300]:
test_model(predict_with_tree, 10, ['age', 'gender','over_budget', 'cuisine_type'], 'rating', tree_type='classifier')

0.04677781340173133

In [301]:
bigfacet(resturants, 'rating','cuisine_type', row='age', hue='over_budget')

/Users/admin/miniconda3/lib/python3.6/site-packages/matplotlib/tight_layout.py:198: UserWarning: tight_layout cannot make axes width small enough to accommodate all axes decorations
  warnings.warn('tight_layout cannot make axes width small enough '


In [303]:
under = resturants[resturants.over_budget == False]
bigfacet(under, 'rating','cuisine_type', row='age', hue='gender')

/Users/admin/miniconda3/lib/python3.6/site-packages/matplotlib/tight_layout.py:198: UserWarning: tight_layout cannot make axes width small enough to accommodate all axes decorations
  warnings.warn('tight_layout cannot make axes width small enough '
